In [1]:
import numpy as np 
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import dataloader as da

In [2]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120

        # Layer 1
        self.conv1 = nn.Conv2d(1, 25, kernel_size = (1, 5))
        self.conv2 = nn.Conv2d(25, 50, kernel_size = (2, 1))
        self.batchnorm1 = nn.BatchNorm2d(50, eps = 1e-05, momentum=0.1)
        self.pooling1 = nn.MaxPool2d(kernel_size=(1, 2))
        
        # Layer 2
        self.conv3 = nn.Conv2d(50, 100, kernel_size = (1, 5))
        self.batchnorm2 = nn.BatchNorm2d(100, eps = 1e-05, momentum=0.1)
        self.pooling2 = nn.MaxPool2d(kernel_size=(1, 2))
        
        # Layer 3
        self.conv4 = nn.Conv2d(100, 200, kernel_size = (1, 5))
        self.batchnorm3 = nn.BatchNorm2d(200, eps = 1e-05, momentum=0.1)
        self.pooling3 = nn.MaxPool2d(kernel_size=(1, 2))        
        
        # Layer 4
        self.conv5 = nn.Conv2d(200, 400, kernel_size = (1, 5))        
        self.batchnorm4 = nn.BatchNorm2d(400, eps = 1e-05, momentum=0.1)
        self.pooling4 = nn.MaxPool2d(kernel_size=(1, 2))             
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(in_features = 17200, out_features = 1, bias = True)
        
    def forward(self, x):

        # Layer 1
        x = self.conv1(x)
        x = self.conv2(x)        
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = self.pooling1(x)
        x = F.dropout(x, 0.5)
        
        # Layer 2
        x = self.conv3(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.pooling2(x)
        x = F.dropout(x, 0.5)
        
        # Layer 3
        x = self.conv4(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        x = self.pooling3(x)      
        x = F.dropout(x, 0.5)
        
        # Layer 4
        x = self.conv5(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        x = self.pooling4(x)      
        x = F.dropout(x, 0.5)

        # FC Layer
        x = x.view(-1, 17200)
        x = self.fc1(x)
        x = F.sigmoid(x)

        return x

In [3]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 64
    
    predicted = []
    
    for i in range(int(len(X)/batch_size)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [4]:
X_train, y_train, X_test, y_test = da.read_bci_data()


(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


In [5]:
batch_size = 64
net = EEGNet().cuda(0)
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())
train_output = []
test_output = []

for epoch in range(300):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e]).float()
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        train_output += outputs.T.tolist()[0]

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data
    
    for i in range(int(len(X_test)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_test[s:e]).float()
        labels = torch.FloatTensor(np.array([y_test[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        test_output += outputs.T.tolist()[0]


    train_accuracy = accuracy_score(np.array(y_train[0:960]), np.array(train_output).round())        
    train_output = []
    test_accuracy = accuracy_score(np.array(y_test[0:960]), np.array(test_output).round())    
    test_output = []
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params) 
    print("Training Loss ", running_loss)
    print("train_accuracy ", train_accuracy*100, "%")
    print("test_accuracy ", test_accuracy*100, "%")   
#     print("Train - ", evaluate(net, X_train, y_train, params))
#     print("Validation - ", evaluate(net, X_val, y_val, params))
#     print("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0


/root/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


['acc', 'auc', 'fmeasure']
Training Loss  tensor(19.2639, device='cuda:0')
train_accuracy  52.604166666666664 %
test_accuracy  61.5625 %

Epoch  1
['acc', 'auc', 'fmeasure']
Training Loss  tensor(10.7573, device='cuda:0')
train_accuracy  65.625 %
test_accuracy  61.77083333333333 %

Epoch  2
['acc', 'auc', 'fmeasure']
Training Loss  tensor(10.7323, device='cuda:0')
train_accuracy  67.39583333333333 %
test_accuracy  70.41666666666667 %

Epoch  3
['acc', 'auc', 'fmeasure']
Training Loss  tensor(9.0887, device='cuda:0')
train_accuracy  72.91666666666666 %
test_accuracy  69.89583333333333 %

Epoch  4
['acc', 'auc', 'fmeasure']
Training Loss  tensor(9.2787, device='cuda:0')
train_accuracy  71.5625 %
test_accuracy  72.5 %

Epoch  5
['acc', 'auc', 'fmeasure']
Training Loss  tensor(8.7727, device='cuda:0')
train_accuracy  72.8125 %
test_accuracy  68.02083333333333 %

Epoch  6
['acc', 'auc', 'fmeasure']
Training Loss  tensor(9.3261, device='cuda:0')
train_accuracy  71.45833333333333 %
test_accur